# **This notebook contain the whole pipeline of review model**

**code for generating the pharases using yake**

In [86]:
!pip install -U yake

In [1]:
import pandas as pd
import numpy as np
import nltk 
import pickle
from tqdm import tqdm
from difflib import SequenceMatcher

In [88]:
data=pd.read_csv("/content/drive/MyDrive/review_api/review-model-api-main/douglas_reviews_sentences_2022-03-21.csv")
data.shape

(915904, 7)

In [366]:
data.head(2)

,r_id,p_e_id,sentence,mixed,neutral,negative,positive,phrases
0,-9223328818815898130,9041810293446515692,the hair feels well cleaned yet well cared for.,0.000178,0.000738,0.000038,0.999046,['hair feels well cleaned']
1,-9223328818815898130,9041810293446515692,the hair color stays bright for a long time.,0.000216,0.001654,0.001242,0.996889,"['hair color stays bright', 'bright for a long..."


##initializing the yake

In [90]:
#@title Initializing the yake { form-width: "1%" }
import yake

language = "en"
max_ngram_size =5
deduplication_threshold = 0.3
numOfKeywords = 5
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None,)


In [91]:
#@title code to get phrases  { form-width: "1%" }
from tqdm import tqdm

def get_phrases(df):
  keywords=[]
  for i in tqdm(df["sentence"]):
    keyword = custom_kw_extractor.extract_keywords(i)
    kwds=[]
    if len(keyword)==0:
      keywords.append(["no phrase"])
    else:
      for l in keyword:
        if len(l[0].split(" "))>1:
          kwds.append(l[0])
      keywords.append(kwds)
  return keywords


def get_unigram_phrases(df):
  keywords=[]
  for i in tqdm(df["sentence"]):
    keyword = custom_kw_extractor.extract_keywords(i)
    kwds=[]
    if len(keyword)==0:
      keywords.append(["no phrase"])
    else:
      for l in keyword:
        kwds.append(l[0])
      keywords.append(kwds)
  return keywords

In [14]:
test_data=data[:10]

In [ ]:
##select based on both
data["phrases"]=get_unigram_phrases(data)
data["phrases"]=get_phrases(data)

In [357]:
test_data.head(1)

,r_id,p_e_id,sentence,mixed,neutral,negative,positive,phrases
0,-9223328818815898130,9041810293446515692,The hair feels well cleaned yet well cared for.,0.000178,0.000738,0.000038,0.999046,"[hair feels, feels well cleaned]"


In [30]:
# data.to_csv("/content/drive/MyDrive/review_api/review-model-api-main/processed_csv/unigram_phrases.csv",index=None)

## **now to generate the tags from the phrases**

In [223]:
data=pd.read_csv("/content/drive/MyDrive/review_api/review-model-api-main/p_id_data.csv")
data.shape

(915904, 8)

In [224]:
data.head(2)

,r_id,p_e_id,sentence,mixed,neutral,negative,positive,phrases
0,-9223328818815898130,9041810293446515692,the hair feels well cleaned yet well cared for.,0.000178,0.000738,0.000038,0.999046,['hair feels well cleaned']
1,-9223328818815898130,9041810293446515692,the hair color stays bright for a long time.,0.000216,0.001654,0.001242,0.996889,"['hair color stays bright', 'bright for a long..."


In [226]:
#@title get stopwords and most frequent unigrams { form-width: "1%" }

import re


def get_stop_words(stop_file_path):
    """load stop words """
    
    with open("/content/drive/MyDrive/review_api/review-model-api-main/stopwords.txt", 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return frozenset(stop_set)

#load a set of stop word
stopwords=get_stop_words("resources/stopwords.txt")

len(stopwords)



def get_top_unigrams():
  with open("/content/drive/MyDrive/review_api/review-model-api-main/processed_csv/unwanted_unigrams.txt", 'r', encoding="utf-8") as f:
      needed_unigrams = f.readlines()
      needed_unigrams = list(m.strip() for m in needed_unigrams)
  needed_unigrams=needed_unigrams[:400]
  return needed_unigrams

needed_unigrams=get_top_unigrams()



398

In [227]:
#@title splitting code { form-width: "1%" }
def splitting(text):
  kwds=[]
  spl=text.split(",")
  length=len(spl)
  if length > 5:
    length=5
  if length>0:
    spl=spl[:length]
    for i in spl:
      i=i.replace("'","")
      i=i.replace("]","")
      i=i.replace("[","")
      i=i.replace("\"","")
      i=i.strip()
      i=i.lower()
      kwds.append(i)
  else:
    kwds.append("empty")
  return kwds 



In [363]:
df=data.copy()
df.head(2)

,r_id,p_e_id,sentence,mixed,neutral,negative,positive,phrases
0,-9223328818815898130,9041810293446515692,the hair feels well cleaned yet well cared for.,0.000178,0.000738,0.000038,0.999046,['hair feels well cleaned']
1,-9223328818815898130,9041810293446515692,the hair color stays bright for a long time.,0.000216,0.001654,0.001242,0.996889,"['hair color stays bright', 'bright for a long..."


In [230]:
df["phrases"]=[splitting(x) for x in df["phrases"]]

In [233]:
#@title find unigrams with no bigrams { form-width: "1%" }
def find_unigrams_with_no_bigrams(df):
  needed_unigrams={}
  for i in tqdm(df["phrases"]):
    if len(i)==1:
      needed_unigrams.update({i[0]:1})
    elif len(i)>1:
      flag=0
      for j in i:
        # print(j)
        if len(j.split(" "))>1:
          flag=1
          break
      if flag==0:
        needed_unigrams.update({i[0]:1})
  return needed_unigrams


    
needed_unigrams=find_unigrams_with_no_bigrams(df)
len(needed_unigrams)

100%|██████████| 915904/915904 [00:01<00:00, 854339.21it/s]


162863

In [234]:
list(needed_unigrams.items())[:10]

[('hair feels well cleaned', 1),
 ('fragrance for a long time', 1),
 ('long time', 1),
 ('enthusiastic about this fragrance', 1),
 ('nice as a gift', 1),
 ('great product and great scent', 1),
 ('love the heat balm', 1),
 ('buy recommendation', 1),
 ('retains its liveliness', 1),
 ('', 1)]

In [137]:
count=0
for i in df["phrases"]:
  if len(i)>4:
    count+=1
print(count)


0


In [140]:
#@title add top 5 phrases { form-width: "1%" }

    
def add_phrases(df):
  ph1,ph2,ph3,ph4,ph5=[],[],[],[],[]
  lists={0:ph1,1:ph2,2:ph3,3:ph4,4:ph5}
  for ph in tqdm(df["phrases"]):
    length=len(ph)
    for i in range(length):
      lists[i].append(ph[i])
    for j in range(length,5):
      lists[j].append("")
  print(len(ph1),len(ph2),len(ph3),len(ph4),len(ph5))
  df["ph1"]=ph1
  df["ph2"]=ph2
  df["ph3"]=ph3
  df["ph4"]=ph4
  df["ph5"]=ph5

  return df

df=add_phrases(df.copy())

100%|██████████| 915904/915904 [00:01<00:00, 503017.43it/s]


915904 915904 915904 915904 915904


In [236]:
df.head(1)

,r_id,p_e_id,sentence,mixed,neutral,negative,positive,phrases
0,-9223328818815898130,9041810293446515692,the hair feels well cleaned yet well cared for.,0.000178,0.000738,0.000038,0.999046,[hair feels well cleaned]


In [142]:
#@title clean the phrases { form-width: "1%", display-mode: "both" }
def clean_phrase(phrases):
  all_pharases=[]
  for i in phrases:
    for j in i:
      if j!="empty" and len(j.split(" "))>0 and j!=" ":
        all_pharases.append(j)
  return all_pharases


In [237]:
all_pharases=clean_phrase(df["phrases"])
len(all_pharases)

1765841

In [238]:
all_pharases=[i for i in all_pharases if len(i.split(" "))>1]
len(all_pharases)

1657648

In [239]:
#@title find unigrams and bigrams { form-width: "1%" }
def find_unigram(lis):
  vocab={}
  sorted_dict = {}
  for phrase in tqdm(lis):
    words=phrase.split(" ")
    for word in words:
      if word in needed_unigrams:
        if word not in stopwords and len(word)>2 and word!=" " and word!="":
          if word not in vocab:
            vocab.update({word:1})
          else:
            vocab.update({word:vocab[word]+1})
  
  vocab_key=sorted(vocab, key=vocab.get,reverse=True) 
  for w in vocab_key:
    sorted_dict[w] = vocab[w]
  return sorted_dict


def find_bigram(lis):
  vocab={}
  sorted_dict = {}
  for phrase in tqdm(lis):
    words=phrase.split(" ")
    words=[i for i in words]
    for index in range(len(words)-1):
      word=" ".join(words[index:index+2])
      if word not in vocab:
          vocab.update({word:1})
      else:
          vocab.update({word:vocab[word]+1})
  vocab_key=sorted(vocab, key=vocab.get,reverse=True) 
  for w in vocab_key:
    sorted_dict[w] = vocab[w]
  return sorted_dict 




  


In [240]:
unigrams=find_unigram(all_pharases)
bigrams=find_bigram(all_pharases)

100%|██████████| 1657648/1657648 [00:06<00:00, 241088.48it/s]


In [241]:
list(bigrams.items())[:10]

[('easy to', 32819),
 ('the skin', 27140),
 ('submitted after', 24152),
 ('after receiving', 23922),
 ('an incentive', 23757),
 ('receiving an', 23728),
 ('long time', 21062),
 ('on the', 20596),
 ('free trial', 19406),
 ('to apply', 19173)]

# **bigram after removing stop words**

In [243]:
#@title clean the unigrams and bigrams { form-width: "1%" }
def clean_bigrams(bigrams):
  print("length of bigrams : ",len(bigrams))
  cp=bigrams.copy()
  for i in cp:
    flag=1
    l=i.split(" ")
    for j in l:
      if j in stopwords:
        flag=0
    if flag==0:
      bigrams.pop(i)
  print("length of bigrams : ",len(bigrams))
  return bigrams

def clean_unigrams(unigrams):
  print("length of unigrams : ",len(unigrams))
  cp=unigrams.copy()
  for i in cp:
    if i in stopwords:
      unigrams.pop(i)
  print("length of unigrams : ",len(unigrams))
  return unigrams

bigrams=clean_bigrams(bigrams)
unigrams=clean_unigrams(unigrams)

length of bigrams :  418497
length of bigrams :  196127
length of unigrams :  0
length of unigrams :  0


In [244]:
list(bigrams.items())[:10]

[('long time', 21062),
 ('free trial', 19406),
 ('skin feels', 8761),
 ('pleasant scent', 7157),
 ('quickly absorbed', 6553),
 ('super easy', 6294),
 ('hair feels', 5892),
 ('dry skin', 5338),
 ('absorbs quickly', 5121),
 ('product test', 4940)]

In [246]:
secondary_bigrams=bigrams.copy()
len(secondary_bigrams)

196127

In [154]:
pip install -U simplemma 

## **To merge the tags having same meaning**

In [248]:
#@title code for lemmatization  { form-width: "1%" }
import simplemma
langdata = simplemma.load_data('en')
def lemmatize(corpus):
  lmt={}
  for sent in corpus:
    words=sent.split(" ")
    words=[simplemma.lemmatize(w,langdata) for w in words]
    lmt.update({sent:words})
  return lmt




In [ ]:
lemmatized=lemmatize(bigrams)
list(lemmatized.items())[:10]

In [250]:
lemmatized_keys=list(lemmatized.keys())

In [251]:
lemmatized_keys=lemmatized_keys[:60000]

In [252]:
#@title code for merge tags  { form-width: "1%" }
def merge_tags(lemmatized,lemmatized_keys):
  cluster=[]
  for index in tqdm(range(len(lemmatized_keys)-1)):
    temp_cluster=[]
    temp_cluster.append(lemmatized_keys[index])
    for test in lemmatized_keys[index+1:]:
      check_words=lemmatized[test]
      score=[]
      for word in lemmatized[lemmatized_keys[index]]:
        if word in check_words:
          score.append("T")
          check_words.remove(word)
        else:
          for test_word in check_words:
            if word[0]==test_word[0]:
              ratio=SequenceMatcher(a=word , b=test_word ).ratio()
              if ratio >.85:
                score.append("T")
        
      if len(score)>1:
        temp_cluster.append(test)
    if len(temp_cluster)>1:
      cluster.append(temp_cluster)  

  return cluster

def merge_tags_unigrams(lemmatized,lemmatized_keys):
  cluster=[]
  for index in tqdm(range(len(lemmatized_keys)-1)):
    temp_cluster=[]
    temp_cluster.append(lemmatized_keys[index])
    for test in lemmatized_keys[index+1:]:
      check_words=lemmatized[test]
      score=[]
      for word in lemmatized[lemmatized_keys[index]]:
         for test_word in check_words:
          ratio=SequenceMatcher(a=word , b=test_word ).ratio()
          if ratio >.85:
            temp_cluster.append(test)
    if len(temp_cluster)>1:
      cluster.append(temp_cluster)
  return cluster





def vocab_merger(vocab,clusters):
  print("len of vocab before merging : ",len(vocab))
  for clst in clusters:
    for keys in clst[1:]:
      vocab[clst[0]]+=vocab[keys]
      vocab.pop(keys)
  print("len of vocab after merging : ",len(vocab))

  return vocab
        

    
def get_replacement_dict(vocab,clusters):
  replacement_dict={}
  for v in vocab:
    replacement_dict.update({v:v})  
  for clst in clusters:
    for keys in clst:
      replacement_dict.update({keys:clst[0]})
      
  print(len(replacement_vocab))
  return replacement_dict





In [194]:
clusters=merge_tags(lemmatized,lemmatized_keys)

100%|██████████| 6984/6984 [00:46<00:00, 149.48it/s] 


In [253]:
#@title Reading and writing the clusters into pickle file { form-width: "1%" }
# with open("/content/drive/MyDrive/review_api/review-model-api-main/processed_csv/bi_gram_cluster.pickle", "wb") as fp:   #Pickling
#   pickle.dump(clusters,fp)

with open("/content/drive/MyDrive/review_api/review-model-api-main/processed_csv/bi_gram_cluster.pickle", "rb") as fp:   # Unpickling
   clusters = pickle.load(fp)


In [254]:
len(clusters)

155

In [262]:
count=0
for i in clusters:
  count+=len(i)-1
print(count)

292


In [279]:
df.head(2)

,r_id,p_e_id,sentence,mixed,neutral,negative,positive,phrases,sent_tag
0,-9223328818815898130,9041810293446515692,the hair feels well cleaned yet well cared for.,0.000178,0.000738,0.000038,0.999046,[hair feels well cleaned],positive
1,-9223328818815898130,9041810293446515692,the hair color stays bright for a long time.,0.000216,0.001654,0.001242,0.996889,"[hair color stays bright, bright for a long time]",positive


In [278]:
#@title add column wise phrases  { form-width: "1%" }
def add_phrases(df):
  ph1,ph2,ph3,ph4,ph5=[],[],[],[],[]
  lists={0:ph1,1:ph2,2:ph3,3:ph4,4:ph5}
  for ph in tqdm(df["phrases"]):
    length=len(ph)
    for i in range(length):
      lists[i].append(ph[i])
    for j in range(length,5):
      lists[j].append("")
  print(len(ph1),len(ph2),len(ph3),len(ph4),len(ph5))
  df["ph1"]=ph1
  df["ph2"]=ph2
  df["ph3"]=ph3
  df["ph4"]=ph4
  df["ph5"]=ph5

  return df

In [280]:
df=add_phrases(df)

100%|██████████| 915904/915904 [00:03<00:00, 271084.53it/s]


915904 915904 915904 915904 915904


In [263]:
clusters;

In [264]:
 vocab=dict(list(bigrams.items()))#[:80000])
 len(vocab)

196127

In [265]:
replacement_vocab=get_replacement_dict(vocab,clusters)
# replacement_vocab

196127


In [267]:
merge_vocab=vocab_merger(vocab,clusters)

len of vocab before merging :  196127
len of vocab after merging :  195835


In [ ]:
replacement_vocab;

In [314]:
needed_unigrams;

In [ ]:
#@title to save the files { form-width: "1%" }
#saving the vocab
import pickle

# with open("/content/drive/MyDrive/review_api/review-model-api-main/processed_csv/tags_freq.pickle", "wb") as fp:   #Pickling
#   pickle.dump(merge_vocab, fp)


# with open("/content/drive/MyDrive/review_api/review-model-api-main/processed_csv/unigram_replacement.pickle", "wb") as fp:   #Pickling
#   pickle.dump(replacement_vocab, fp)




In [ ]:
#@title to read the files { form-width: "1%" }
# with open("/content/drive/MyDrive/review_api/review-model-api-main/processed_csv/bigrams_tags_freq.pickle", "rb") as fp:   # Unpickling
#    merge_vocab = pickle.load(fp)

# with open("/content/drive/MyDrive/review_api/review-model-api-main/processed_csv/unigram_replacement.pickle", "rb") as fp:   # Unpickling
#    unigram_replacement_vocab = pickle.load(fp)
  
# with open("/content/drive/MyDrive/review_api/review-model-api-main/processed_csv/bigrams_replacement.pickle", "rb") as fp:   # Unpickling
#    bigram_replacement_vocab = pickle.load(fp)



print(len(merge_vocab))
print(len(unigram_replacement_vocab))
print(len(bigram_replacement_vocab))

In [364]:
#@title  to add unigrams along with bigrams { form-width: "1%" }
def get_unigrams(unigram_replacement_vocab,needed_unigrams):
  un=unigram_replacement_vocab.copy()
  print(len(unigram_replacement_vocab))
  for i in un:
    if i not in needed_unigrams:
      unigram_replacement_vocab.pop(i)
  len(unigram_replacement_vocab)
  return unigram_replacement_vocab

unigram_replacement_vocab= get_unigrams(unigram_replacement_vocab,needed_unigrams)


400


400

In [342]:
len(needed_unigrams)

400

In [269]:

vocab_1=dict(list(vocab.items())[:40000])

replacement_vocab_1={}
for i in vocab_1:
  if i not in  replacement_vocab:
    replacement_vocab_1.update({i:i})
  else:
    replacement_vocab_1.update({i:replacement_vocab[i]})
len(replacement_vocab_1)

40000

In [216]:
merge_vocab;

In [275]:
df.columns

Index(['r_id', 'p_e_id', 'sentence', 'mixed', 'neutral', 'negative',
       'positive', 'phrases', 'sent_tag'],
      dtype='object')

In [ ]:
#@title creating a overall dataframe { form-width: "1%" }
df["sent_tag"]=df[['neutral', 'negative', 'positive']].idxmax(axis=1)
f1=df[['r_id', 'p_e_id', 'sentence', 'neutral', 'negative',
       'positive', 'phrases', 'ph1','sent_tag']]
print(df1.shape)


df2=df[['r_id', 'p_e_id', 'sentence', 'neutral', 'negative',
       'positive', 'phrases', 'ph2','sent_tag']]

print(df2.shape)

df3=df[['r_id', 'p_e_id', 'sentence', 'neutral', 'negative',
       'positive', 'phrases', 'ph3','sent_tag']]

print(df3.shape)

df4=df[['r_id', 'p_e_id', 'sentence', 'neutral', 'negative',
       'positive', 'phrases', 'ph4','sent_tag']]


print(df4.shape)

df5=df[['r_id', 'p_e_id', 'sentence', 'neutral', 'negative',
       'positive', 'phrases', 'ph5','sent_tag']]

print(df5.shape)


df1.rename(columns = {'ph1':'ph'}, inplace = True)
df2.rename(columns = {'ph2':'ph'}, inplace = True)
df3.rename(columns = {'ph3':'ph'}, inplace = True)
df4.rename(columns = {'ph4':'ph'}, inplace = True)
df5.rename(columns = {'ph5':'ph'}, inplace = True)


dfs=pd.concat([df1,df2,df3,df4,df5])
dfs.shape

In [290]:
#@title preprocessing on df , removing null values { form-width: "1%" }
dfs=dfs.replace(r'^\s*$', np.nan, regex=True)
dfd = dfs[dfs['ph'].notna()]
dfd["phrases"]=dfd["ph"]
dfd.shape

,r_id,p_e_id,sentence,neutral,negative,positive,phrases,ph,sent_tag
0,-9223328818815898130,9041810293446515692,the hair feels well cleaned yet well cared for.,0.000738,0.000038,0.999046,[hair feels well cleaned],hair feels well cleaned,positive
1,-9223328818815898130,9041810293446515692,the hair color stays bright for a long time.,0.001654,0.001242,0.996889,"[hair color stays bright, bright for a long time]",hair color stays bright,positive
2,-9222828304918768305,5153922226071839655,"we have known the fragrance for a long time, a...",0.012318,0.000189,0.987403,[fragrance for a long time],fragrance for a long time,positive
3,-9222828304918768305,5153922226071839655,it lasts a long time.,0.002139,0.000762,0.997045,[long time],long time,positive
4,-9222825686026871200,-683321389213928801,i am very enthusiastic about this fragrance.,0.002101,0.000742,0.997105,[enthusiastic about this fragrance],enthusiastic about this fragrance,positive


In [293]:
dfd.columns

Index(['r_id', 'p_e_id', 'sentence', 'neutral', 'negative', 'positive',
       'phrases', 'ph', 'sent_tag'],
      dtype='object')

In [294]:
new_df=pd.DataFrame({"r_id":dfd["r_id"],"p_e_id":dfd["p_e_id"],"sent_neu_score":dfd["neutral"],"sent_neg_score":dfd["negative"],"sent_pos_score":dfd["positive"],"phrases":dfd["phrases"],"sent_tag":dfd["sent_tag"]})
new_df.head()

In [ ]:
new_df.to_csv("/content/drive/MyDrive/review_api/review-model-api-main/processed_csv/phrases.csv",index=None)

In [ ]:
new_df.shape

(1765969, 7)

In [ ]:
new_replacement_vocab=replacement_vocab.copy()

In [296]:
#@title updating replacement vocab { form-width: "10%" }
def update_replacement_vocab(bigrams,replacement_vocab)
  for i in bigrams:
    if i not in replacement_vocab:
      replacement_vocab.update({i:i})
  len(replacement_vocab)
  return replacement_vocab

replacement_vocab=update_replacement_vocab(bigrams,replacement_vocab)

196127

In [344]:
#@title code to gave tags to phrases { form-width: "1%" }
from tqdm import tqdm

def give_tag(new_df,replacement_vocab,replacement_vocab_1):
  # print("ajay")
  tags=[]
  for ph in tqdm(new_df["phrases"]):
    flag=0  
    if flag==0:
      spl=ph.split(" ")
      for i in range(len(spl)-1):
        bigram=" ".join(spl[i:i+2])
        if bigram in replacement_vocab_1:
          flag=1
          tags.append(replacement_vocab_1[bigram])
          break
    if flag==0:
      spl=ph.split(" ")
      if flag==0:
        for i in range(len(spl)-1):
          bigram=" ".join(spl[i:i+2])
          if bigram in replacement_vocab:
            flag=1
            tags.append(replacement_vocab[bigram])
            break

    if flag==0:
      spl=ph.split(" ")
      if flag==0:
        for i in range(len(spl)):
          unigram=spl[i]
          if unigram in unigram_replacement_vocab:
            flag=1
            tags.append(unigram_replacement_vocab[unigram])
            # print(unigram)
            break
    if flag==0:
      tags.append("")

  print(len(tags))
  return tags



from tqdm import tqdm

def give_tag_(new_df,replacement_vocab):
  tags=[]
  for ph in tqdm(new_df["phrases"]):
    flag=0
    spl=ph.split(" ")
    if flag==0:
      for i in range(len(spl)-1):
        bigram=" ".join(spl[i:i+2])
        if bigram in replacement_vocab:
          flag=1
          tags.append(replacement_vocab[bigram])
          break
    if flag==0:

      tags.append("")

  print(len(tags))
  return tags

tags=gave_tag(new_df,replacement_vocab,replacement_vocab_1)

new_df["phrase_norm"]=tags

100%|██████████| 1657648/1657648 [00:08<00:00, 195037.29it/s]

1657648


In [350]:
#@title preprocessing on final df { form-width: "2%" }
ddff=new_df.copy()
ddff=ddff.replace(r'^\s*$', np.nan, regex=True)
ddff = ddff[ddff['phrase_norm'].notna()]
ddff=ddff.reset_index(drop=True)
ddff.head()

In [352]:
ddff.head()

,r_id,p_e_id,sent_neu_score,sent_neg_score,sent_pos_score,phrases,sent_tag,phrase_norm
0,-9223328818815898130,9041810293446515692,0.000738,0.000038,0.999046,hair feels well cleaned,positive,hair feels
1,-9223328818815898130,9041810293446515692,0.001654,0.001242,0.996889,hair color stays bright,positive,hair color
2,-9222828304918768305,5153922226071839655,0.012318,0.000189,0.987403,fragrance for a long time,positive,long time
3,-9222828304918768305,5153922226071839655,0.002139,0.000762,0.997045,long time,positive,long time
4,-9222825686026871200,-683321389213928801,0.002101,0.000742,0.997105,enthusiastic about this fragrance,positive,enthusiastic


In [353]:
ddff.shape

(1437057, 8)

In [355]:
#@title saving the final output { form-width: "1%" }
#ddff.to_csv("/content/drive/MyDrive/review_api/review-model-api-main/processed_csv/final_output.csv",index=None)

## **testing the output**

In [2]:
df=pd.read_csv("/content/drive/MyDrive/review_api/review-model-api-main/processed_csv/final_output.csv")


In [3]:
df

,r_id,p_e_id,sent_neu_score,sent_neg_score,sent_pos_score,phrases,sent_tag,phrase_norm
0,-9223328818815898130,9041810293446515692,0.000738,0.000038,0.999046,hair feels well cleaned,positive,hair feels
1,-9223328818815898130,9041810293446515692,0.001654,0.001242,0.996889,hair color stays bright,positive,hair color
2,-9222828304918768305,5153922226071839655,0.012318,0.000189,0.987403,fragrance for a long time,positive,long time
3,-9222828304918768305,5153922226071839655,0.002139,0.000762,0.997045,long time,positive,long time
4,-9222825686026871200,-683321389213928801,0.002101,0.000742,0.997105,enthusiastic about this fragrance,positive,enthusiastic
...,...,...,...,...,...,...,...,...
1437052,9222526729149160970,-495150830619648880,0.013807,0.001284,0.983931,skin is rosier,positive,skin
1437053,9222470566337421148,2250777992411312234,0.006035,0.055188,0.931771,reacts very much with impurities,positive,impurities
1437054,9221648364451493270,-2346029291979487605,0.025258,0.016751,0.913630,treatment stays,positive,treatment stays
1437055,9223277598457306173,-644069247751601838,0.007064,0.000321,0.992458,smells so good,positive,smells


In [4]:
df["phrase_norm"].value_counts()

easy                 26135
skin                 22809
hair                 21593
scent                21516
free trial           19406
                     ...  
dandelion cake           1
non-slip profile         1
three-four hours         1
manhattan locarno        1
treatment stays          1
Name: phrase_norm, Length: 139234, dtype: int64

## **few DUCK DB commands**

In [5]:
!pip install duckdb==0.3.2

     |████████████████████████████████| 12.5 MB 4.5 MB/s 


In [6]:
import duckdb
# to start an in-memory database
con = duckdb.connect(database=':memory:')
# to use a database file (not shared between processes)
con = duckdb.connect(database='my-db.duckdb', read_only=False)
# to use a database file (shared between processes)
con = duckdb.connect(database='my-db.duckdb', read_only=True)

In [7]:
df.to_parquet("output_packet.parquet")

In [ ]:
SELECT * FROM read_parquet('test.parquet');

In [8]:
con.register('test_df_view', df)

In [12]:
con.execute("SELECT phrase_norm,count(*)\
            FROM test_df_view\
            GROUP BY phrase_norm")

In [13]:
con.fetchall()

[('hair feels', 5772),
 ('hair color', 1368),
 ('nice', 3717),
 ('expectations', 636),
 ('fit mai', 81),
 ('heat balm', 2),
 ('immediately rises', 2),
 ('buy recommendation', 812),
 ('eyebrow pencil', 670),
 ('birthday', 347),
 ('application', 4463),
 ('quickly absorbed', 11558),
 ('skin feels', 8962),
 ('higher-priced serum', 1),
 ('short eyelashes', 176),
 ('price performance', 354),
 ('mascara takes', 4),
 ('perfume', 5896),
 ('hair structure', 339),
 ('hair', 21593),
 ('hair looks', 1834),
 ('finish', 670),
 ('super easy', 5987),
 ('facial skin', 711),
 ('beard hair', 147),
 ('smells', 10087),
 ('lips', 3310),
 ('retro dark', 3),
 ('color result', 663),
 ('reddened skin', 47),
 ('eyelash curler', 269),
 ('single product', 29),
 ('recently received', 105),
 ('tart wooden', 1),
 ('moisture', 2054),
 ('brush', 4156),
 ('hairspray', 788),
 ('shine', 1972),
 ('sexy scent', 32),
 ('compliments', 913),
 ('women', 739),
 ('perfume smells', 451),
 ('travel size', 94),
 ('thin eyelashes', 38

In [10]:
con.execute("SELECT count(*)\
            FROM test_df_view")

In [11]:
con.fetchall()

[(1437057,)]